# Pre processing of the dataset

### Importing Required modules

In [1]:
import json
import csv
import pandas as pd
import sys
import fileinput

### Removal of "NumberInt" data type from the orginial dataset

In [2]:

f = open(r"C:\Users\User\Downloads\dblpv13.json", "r+",encoding='utf-8')
f2=open(r"C:/Users/User/Downloads/capstone_1gb.json", "a+",encoding='utf-8')
while True:
    line=f.readline()
    if('NumberInt' in line):        
        #print(len(line),line)
        if("n_citation" in line):
            line1=line.removesuffix('), \n')
            line_new=line1[0:19:]+ "\""+line1[29::]+"\"," '\n'
            f2.writelines(line_new)
            
        elif("year" in line):
            line1=line.removesuffix('), \n')
            line_new=line1[0:13:]+ "\""+line1[23::]+"\"," '\n'
            f2.writelines(line_new)
        elif("type" in line):
            if (',' in line ):
                line1=line.removesuffix('), \n')
                line_new=line1[0:17:]+ "\""+line1[28::]+"\"," '\n'
                f2.writelines(line_new)
            else:
                line1=line.removesuffix(')\n')
                line_new=line1[0:17:]+ "\""+line1[28::]+"\"" '\n'
                f2.writelines(line_new)
    else:
        f2.writelines(line)
    if not line:
        break 


f2.close()

### Loading the json file

In [3]:
f = open(r"C:\Users\User\Downloads\capstone_500mb_1.json",encoding = "utf-8")
data_list = json.load(f)

### Removing papers with no abstract and citations less than 5

In [4]:
cl=[] 
cl1=[]
for i in range(len(data_list)):
    for key in data_list[i]:
        if key=="abstract":
            if data_list[i][key]!="":
                #print(data_list[i])
                cl1.append(data_list[i])
    
for i in range(len(cl1)):
    for key in cl1[i]:
        if key=="n_citation":
            if int(cl1[i][key])>=5:
                cl.append(cl1[i])

### Limiting the papers to particular domains (domain_list) and Removing useless attributes of authors 

In [5]:
fd=[]
domain_list=["pattern recognition","feature recognition","distributed computing","handwritting recognition","cloud computing","artificial intelligence","machine learning","neural networks","robotics","data science","data mining"]
remove_auth_attr = ["org","sid","email","orgid","orgs","gid","oid","_id","orcid","bio"]
for i in range (len(cl)):
    for key in cl[i]:
        if key=="fos" or key=="keywords":
            a=(cl[i][key])
            for word in range(len(a)):
                a[word]=a[word].lower()
            for domain in domain_list:
                if domain in a:
                    if (cl[i]) not in fd:       
                        fd.append(cl[i]) 
        if key == "authors":
            temp = cl[i][key]

            for j in temp:
                for attr in list(j.keys()):
                    if attr in remove_auth_attr:
                        j.pop(attr)

In [12]:
#removing unwanted attributes of venue 
remove_venue_attr=["_id"]
for i in range(len(cl)):
    for key in cl[i]:
        if(key=="venue"):
            temp=cl[i][key]
            for j in temp:
                for attr in list(j.keys()):
                    if attr in remove_venue_attr:
                        j.pop(attr)

AttributeError: 'str' object has no attribute 'keys'

### Changing the data type of authors to lists

In [6]:
i = 0
while i < len(cl):
    names = []
    if "authors" in cl[i]:
        for j in cl[i]["authors"]:
            if "name" in list(j.keys()):
                names.append(j["name"])
        cl[i]["authors"]=names
    i+=1

### Removing unncessary characters

In [7]:
i = 0
while i < len(cl):
    if "title" in cl[i]:
        cl[i]["title"] = cl[i]["title"].replace("'","`")
        cl[i]["title"] = cl[i]["title"].replace("]","")       
        cl[i]["title"] = cl[i]["title"].replace(":","")
        cl[i]["title"] = cl[i]["title"].replace(".","")
        cl[i]["title"] = cl[i]["title"].replace("-","")
    i += 1

### Removing unwanted attributes

In [8]:

useless_attr=['pdf' ,'page_end', 'lang', 'volume', 'isbn', 'page_start', 'issn', 'issue']

for i in fd:
    for j in i.copy():
        if j in useless_attr:
            del i[j]

###  Limiting the number of urls to 1

In [9]:
for i in range (len(fd)):  
    for key in fd[i]:
        if key=="url":                
            fd[i][key]=fd[i][key][0:1]


### Removing the self references 

In [10]:
i=0
while (i<len(fd)):
    if 'references' in fd[i]:
        j=0
        while(j<len(fd[i]['references'])):
            if(fd[i]['_id']==fd[i]['references'][j]):
                fd[i]['references'].pop(j)
                
            j+=1
    i+=1
    

### Saving the preprocessed data as csv file

In [15]:
import csv
keys = ["_id","title","authors","year","keywords","fos","n_citation","doi","url","abstract","references","venue"]


a_file = open(r"C:\Users\User\Desktop\output_500mb_venue.csv", "w",encoding='utf-8')
dict_writer = csv.DictWriter(a_file, keys,extrasaction='ignore')
dict_writer.writeheader()
dict_writer.writerows(fd)
a_file.close()